# 데이터 전처리



In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import os

In [2]:
# 결측치 처리 함수
def replace_NaN (df, col, replacement):
    df[col] = df[col].fillna(replacement)

### product 데이터 전처리 통합

In [3]:
# product table 가져오기
base_path = os.getcwd()

# csv
product = pd.read_csv(base_path + '/product.csv')
product.head()

# 결측치 확인
product.isnull().sum()

id                      0
gender                  0
masterCategory          0
subCategory             0
articleType             0
baseColour             15
season                 21
year                    1
usage                 317
productDisplayName      7
dtype: int64

In [4]:
# 결측치 unknown으로 대체
cols = ['baseColour', 'season', 'year', 'usage', 'productDisplayName']
for col in cols:
    replace_NaN(product, col, 'unknown')

In [5]:
#중복 없음
product.duplicated().sum()

0

In [6]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44446 entries, 0 to 44445
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  44446 non-null  int64 
 1   gender              44446 non-null  object
 2   masterCategory      44446 non-null  object
 3   subCategory         44446 non-null  object
 4   articleType         44446 non-null  object
 5   baseColour          44446 non-null  object
 6   season              44446 non-null  object
 7   year                44446 non-null  object
 8   usage               44446 non-null  object
 9   productDisplayName  44446 non-null  object
dtypes: int64(1), object(9)
memory usage: 3.4+ MB


In [7]:
# 특성 이름 transaction_new 테이블과 통일
product.rename(columns = {'id' : 'product_id'})

,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt
...,...,...,...,...,...,...,...,...,...,...
44441,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013.0,Casual,Gas Men Caddy Casual Shoe
44442,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011.0,Casual,Lotto Men's Soccer Track Flip Flop
44443,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Puma Men Graphic Stellar Blue Tshirt
44444,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017.0,Casual,Rasasi Women Blue Lady Perfume


In [8]:
# productDisplayName에서 첫번째 단어를 브랜드명으로 간주
# 데이터 확인 후 최대한 구분 기준을 세부화했으나 약 3000개 정도 구분점 설정 실패 -> 그냥 첫번째 글자를 브랜드명으로 쓰기로 결정
brand = product.productDisplayName.str.split().str[0]
brand = brand.str.lower()
product['brand'] = brand.str.lower()

In [9]:
#사용하지 않는 특성 제거
product.drop(columns = 'year', inplace = True)

In [10]:
product.to_csv('product_final.csv', index = False)

### click_stream_new 데이터 전처리 통합

In [11]:
click_stream_new = pd.read_csv(base_path + '/click_stream_new.csv')
click_stream_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12833602 entries, 0 to 12833601
Data columns (total 12 columns):
 #   Column           Dtype  
---  ------           -----  
 0   session_id       object 
 1   event_name       object 
 2   event_time       object 
 3   event_id         object 
 4   traffic_source   object 
 5   product_id       float64
 6   quantity         float64
 7   item_price       float64
 8   payment_status   object 
 9   search_keywords  object 
 10  promo_code       object 
 11  promo_amount     float64
dtypes: float64(4), object(8)
memory usage: 1.1+ GB


In [12]:
# 컬럼명 수정
click_stream_new.columns = click_stream_new.columns.str.strip()

In [13]:
# 결측치 확인
click_stream_new.isnull().sum()

session_id                0
event_name                0
event_time                0
event_id                  0
traffic_source            0
product_id         10896445
quantity           10896445
item_price         10896445
payment_status     11981020
search_keywords    11660336
promo_code         12507067
promo_amount       12507067
dtype: int64

In [14]:
#중복값 없음
click_stream_new.duplicated().sum()

0

In [15]:
# 데이터 타입 설정
click_stream_new['event_time'] = pd.to_datetime(click_stream_new['event_time'])

# 기간 조정 (2020-08 ~ 2022-07)
click_stream_new = click_stream_new[(click_stream_new['event_time'] >= '2020-08-01') & (click_stream_new['event_time'] <= '2022-07-31')]

# 결측치 대체
cols = ['quantity', 'item_price', 'promo_amount']
for col in cols:
    replace_NaN(click_stream_new, col, 0)

# promo code 변환 / 0 : promo_code를 사용하지 않은 경우
promo_code_mapping = {'AZ2022': 1, 'BUYMORE': 2, 'WEEKENDSERU': 3, 'XX2022': 4, 'LIBURDONG': 5, 'WEEKENDMANTAP': 6, 'SC2022': 7, 'STARTUP': 8}
click_stream_new['promo_code'] = click_stream_new['promo_code'].map(promo_code_mapping).fillna(0)

# Success :1 / Failed :0 으로 변환 후 카테고리로 변환
click_stream_new['payment_status'] = click_stream_new['payment_status'].map({'Success' : 1, 'Failed' : 0})

# 사용하지 않는 컬럼 제거
click_stream_new.drop(columns = ['event_id'] , inplace = True)

# session_id, 시간 기준으로 오름차순 정렬
click_stream_new.sort_values(by=['session_id', 'event_time'], inplace=True)

In [16]:
# search_keywords 번역
    # 1등 Dress Kondangan -> Dress
    # 2등 Tas Wanita -> 여성용 가방
    # 3등 Bekas -> 중고(*)
    # 5등 Kaos -> 셔츠
    # 6등 Baju -> Blouse
    # 7등 Dress Pesta -> Dress
    # 8등 Sepatu NIKE -> 나이키 구두
    # 9등 Tas Kerja -> 업무용 가방
    # 10등 Second -> 중고(*)
    # 11등 Sepatu -> 구두
    # 12등 Second Hand -> 중고(*)
    # 13등 Thrift -> 중고(*)
    # 14등 Celana Panjang -> 긴 바지
    # 15등 Berang Bekas -> 중고 상품(*)
    # 16등 Sepatu Adidas -> 아디다스 구두
    # 17등 Celana -> 바지
    # 20등 jam -> watch
    # 21등 Topi -> 모자
    # 22등 Baju Tidur -> 잠옷
    # 23등 Daster -> 하우스 드레스

In [17]:
# Search Keywords 번역 적용
search_dict = {'Dress Kondangan':'Dress', 'Tas Wanita':'Woman\'s Bag', 'Bekas':'Second Hand', 'T-shirt':'T-Shirt',
               'Kaos':'Shirts', 'Baju':'Shirts', 'Dress Pesta':'Dress', 'Sepatu Nike':'Nike Shoes', 'Tas Kerja':'Work Bag',
               'Second':'Second Hand', 'Sepatu':'Shoes', 'Thrift':'Second Hand', 'Celana Panjang':'Trousers',
               'Barang Bekas':'Second Hand', 'Sepatu Adidas':'Adidas Shoes', 'Celana':'Trousers', 'Sendal':'Sandal',
               'Jam':'Watch', 'Topi':'Hat', 'Baju Tidur':'Pajamas', 'Daster':'House dress'}
click_stream_new.search_keywords = click_stream_new.search_keywords.replace(search_dict)
click_stream_new.search_keywords.unique()

array([nan, 'Shirts', 'Dress', 'Adidas Shoes', 'Nike Shoes', 'Trousers',
       'Shoes', 'T-Shirt', "Woman's Bag", 'Second Hand', 'Work Bag',
       'Watch', 'Sandal', 'Hat', 'Pajamas', 'House dress'], dtype=object)

In [18]:
click_stream_new.to_csv('click_stream_final.csv', index = False)

### customer 데이터 전처리 통합

In [19]:
# 데이터 로드
customer = pd.read_csv(base_path + '/customer.csv', usecols = ['customer_id', 'gender', 'birthdate', 'home_location', 'first_join_date'])
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   customer_id      100000 non-null  int64 
 1   gender           100000 non-null  object
 2   birthdate        100000 non-null  object
 3   home_location    100000 non-null  object
 4   first_join_date  100000 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [20]:
# null값 없음
customer.isnull().sum()

customer_id        0
gender             0
birthdate          0
home_location      0
first_join_date    0
dtype: int64

In [21]:
# 중복값 없음
customer.duplicated().sum()

0

In [22]:
# 데이터 타입 변경
customer['first_join_date'] = pd.to_datetime(customer['first_join_date'])
customer['birthdate'] = pd.to_datetime(customer['birthdate'])

In [23]:
#가입 일자 기준 나이 & 현재 기준 나이 컬럼 생성
customer['age_join'] = (customer['first_join_date'].dt.year - customer['birthdate'].dt.year).astype('int')
customer['age_now'] = (2022 - customer['birthdate'].dt.year).astype('int')

In [24]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   customer_id      100000 non-null  int64         
 1   gender           100000 non-null  object        
 2   birthdate        100000 non-null  datetime64[ns]
 3   home_location    100000 non-null  object        
 4   first_join_date  100000 non-null  datetime64[ns]
 5   age_join         100000 non-null  int64         
 6   age_now          100000 non-null  int64         
dtypes: datetime64[ns](2), int64(3), object(2)
memory usage: 5.3+ MB


In [25]:
# customer_id 51844 제거 (회원가입은 되어있지만, click_stream_new 테이블에서 session_id가 누락된 고객이기 때문에 제거 결정)
customer = customer.drop(customer[customer['customer_id'] == 51844].index)

In [26]:
customer.to_csv('customer_final.csv', index = False)

### transaction_new 데이터 전처리 통합

In [27]:
transaction_new = pd.read_csv(base_path + '/transaction_new.csv')
transaction_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1254585 entries, 0 to 1254584
Data columns (total 16 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   created_at              1254585 non-null  object 
 1   customer_id             1254585 non-null  int64  
 2   booking_id              1254585 non-null  object 
 3   session_id              1254585 non-null  object 
 4   payment_method          1254585 non-null  object 
 5   payment_status          1254585 non-null  object 
 6   promo_amount            1254585 non-null  int64  
 7   promo_code              481139 non-null   object 
 8   shipment_fee            1254585 non-null  int64  
 9   shipment_date_limit     1254585 non-null  object 
 10  shipment_location_lat   1254585 non-null  float64
 11  shipment_location_long  1254585 non-null  float64
 12  total_amount            1254585 non-null  int64  
 13  product_id              1254585 non-null  int64  
 14  qu

In [28]:
transaction_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1254585 entries, 0 to 1254584
Data columns (total 16 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   created_at              1254585 non-null  object 
 1   customer_id             1254585 non-null  int64  
 2   booking_id              1254585 non-null  object 
 3   session_id              1254585 non-null  object 
 4   payment_method          1254585 non-null  object 
 5   payment_status          1254585 non-null  object 
 6   promo_amount            1254585 non-null  int64  
 7   promo_code              481139 non-null   object 
 8   shipment_fee            1254585 non-null  int64  
 9   shipment_date_limit     1254585 non-null  object 
 10  shipment_location_lat   1254585 non-null  float64
 11  shipment_location_long  1254585 non-null  float64
 12  total_amount            1254585 non-null  int64  
 13  product_id              1254585 non-null  int64  
 14  qu

In [29]:
# promo_code에만 결측치 / 0으로 대체
transaction_new.isnull().sum()

created_at                     0
customer_id                    0
booking_id                     0
session_id                     0
payment_method                 0
payment_status                 0
promo_amount                   0
promo_code                773446
shipment_fee                   0
shipment_date_limit            0
shipment_location_lat          0
shipment_location_long         0
total_amount                   0
product_id                     0
quantity                       0
item_price                     0
dtype: int64

In [30]:
# 중복값 없음
transaction_new.duplicated().sum()

0

In [31]:
# 데이터 타입 변환
transaction_new['created_at'] = pd.to_datetime(transaction_new['created_at'])

# 기간 조정 (2020-08 ~ 2022-07)
transaction_new = transaction_new[(transaction_new['created_at'] >= '2020-08-01') & (transaction_new['created_at'] <= '2022-07-31')]

# Success :1 / Failed :0 으로 변환
transaction_new['payment_status'] = transaction_new['payment_status'].map({'Success' : 1, 'Failed' : 0})

# promo code 변환 / 0 : promo_code를 사용하지 않은 경우
# promo_code_mapping = {'AZ2022': 1, 'BUYMORE': 2, 'WEEKENDSERU': 3, 'XX2022': 4, 'LIBURDONG': 5, 'WEEKENDMANTAP': 6, 'SC2022': 7, 'STARTUP': 8}
transaction_new['promo_code'] = transaction_new['promo_code'].map(promo_code_mapping).fillna(0).astype('category')

# promocode 사용 여부
transaction_new['with_promo'] = transaction_new['promo_code'].apply(lambda x: 1 if x != 0 else 0).astype('category')

# 사용하지 않는 컬럼 제거
transaction_new.drop(columns = ['shipment_date_limit', 'shipment_location_lat', 'shipment_location_long'], inplace = True)

transaction_new['quantity'] = transaction_new['quantity'].astype('int')


In [32]:
transaction_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 781566 entries, 27 to 1254582
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   created_at      781566 non-null  datetime64[ns, UTC]
 1   customer_id     781566 non-null  int64              
 2   booking_id      781566 non-null  object             
 3   session_id      781566 non-null  object             
 4   payment_method  781566 non-null  object             
 5   payment_status  781566 non-null  int64              
 6   promo_amount    781566 non-null  int64              
 7   promo_code      781566 non-null  category           
 8   shipment_fee    781566 non-null  int64              
 9   total_amount    781566 non-null  int64              
 10  product_id      781566 non-null  int64              
 11  quantity        781566 non-null  int64              
 12  item_price      781566 non-null  int64              
 13  with_promo  

In [33]:
# customer_id 51844 제거 
# 해당 transaction_new 테이블에서의 session_id가 click_stream_new 테이블에서 누락되어 제거 결정)
transaction_new = transaction_new.drop(transaction_new[transaction_new['customer_id'] == 51844].index)

In [34]:
transaction_new.to_csv('transaction_final.csv', index = False)